In [1]:
from deepface import DeepFace
from datetime import datetime
import time
import numpy as np

import cv2
import multiprocessing as mp
from multiprocessing import Queue
from queue import Empty, Full

import utils
from utils import *

import importlib
importlib.reload(DeepFace)
importlib.reload(utils)

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import credentials
from credentials import *

In [13]:
from utils import *
importlib.reload(utils)

<module 'credentials' from 'C:\\Users\\Antony\\Documents\\uChicago\\Winter 2022\\Music Thesis\\credentials.py'>

In [7]:
## Frontend: facial recognition video and analyses loops

video_q = Queue(1) # initialize video queue for rendering and emotion analysis. Max size of 1 for pulling data from
data_q = Queue(1) # initialize data queue pipeline. Max size of 1 for real-time data
emotion_q = Queue(1) # same as data queue, but for video

video_loop_mp = mp.Process(target=utils.video_loop, args = (video_q, data_q, emotion_q))
data_loop_mp = mp.Process(target=utils.analysis_loop, args = (video_q, data_q, emotion_q))

video_loop_mp.start()
data_loop_mp.start()

In [76]:
# Emergency stop to the loops if they become runaway

video_loop_mp.terminate()
data_loop_mp.terminate()

In [72]:
creds()

In [5]:
scope = ['user-top-read', 'user-read-playback-state', 'user-modify-playback-state', 'user-read-private']

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, show_dialog = True))

user_top_songs = sp.current_user_top_tracks()
user_top_artists = sp.current_user_top_artists()
    
available_genres = sp.recommendation_genre_seeds()

user_top_genres = []
genre_overlap = []

# Gets user's region so only available songs are played
# TODO: figure out how to do this through the request header
user_country = sp.current_user()['country']

# Gets user's top songs
# Also makes overlap between available genre seeds and user top genres
# Makes a few pretty expensive calls to figure out the genre of each song. Could be removed
user_top_song_uris = []
for track in user_top_songs['items']:
    song_uri = track['uri']
    user_top_song_uris.append(song_uri)
    for artist in track['artists']:
        artist_uri = artist['uri']
        genres = sp.artist(artist_uri)['genres']
        for genre in genres:
            genre = '-'.join(genre.split())
            if genre not in user_top_genres:
                user_top_genres.append(genre)
            if (genre in available_genres['genres']) and (genre not in genre_overlap):
                genre_overlap.append(genre)
    

# Gets user's top genres from top artists
# Also makes overlap between available genre seeds and user top genres
# for artist in user_top_artists['items']:
#     genres = artist['genres']
#     for genre in genres:
#         genre = '-'.join(genre.split())
#         if genre not in user_top_genres:
#             user_top_genres.append(genre)
#         if (genre in available_genres['genres']) and (genre not in genre_overlap):
#             genre_overlap.append(genre)

Enter the URL you were redirected to: https://example.com/callback/?code=AQA-tgA5zexoRcfU6I_BRvhBldoKij6I9NrS767mrmUno6ogJas0wzcsHa7g6QT_VgJ4z_FpFn2vOp7LNweVAeHt_-7yoUIPiAh5lfG_Dz_jIQxtyv6QPuyCL3-5e1Jej_gBoLOF7f-tLpaLnwiYkoCcjhbEy1y8FuwrH7JGJtht3LcuN4x7un7TrjUde_clLYVCa0N8xsSprLRMBaxKSchdzvnP4QSmIWbaKCZT_zEQz8KrX6Qn-cshsCdIC-l8yH6S1unXzm9I5fCgv5WiqJrclyymmww21w


In [65]:
# emotion_labels = ['angry', 'disgusted', 'scared', 'happy', 'sad', 'surprised', 'neutral']

angry_params = {
    'min_acousticness': 0,
    'min_danceability': 0.2,
    'min_energy': 0.4,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
#     'min_speechiness': 0,
    'min_tempo': 80,
    'min_valence': 0,
    
    'max_acousticness': 0.8,
    'max_danceability': 0.65,
    'max_energy': 1,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
#     'max_tempo': 120,
    'max_valence': 0.6,
    
    'target_acousticness': 0.3,
    'target_danceability': 0.4,
    'target_energy': 0.65,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.2,
    'target_tempo': 140,
    'target_valence': 0.2    
}

disgusted_params = {
    'min_acousticness': 0,
    'min_danceability': 0.2,
    'min_energy': 0.4,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.6,
    'max_energy': 0.7,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 0.5,
    
#     'target_acousticness': 0.5,
    'target_danceability': 0.4,
    'target_energy': 0.55,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.2
}

scared_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.4,
    'max_energy': 0.5,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
#     'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 0.5,
    'max_tempo': 120,
    'max_valence': 0.6,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.2,
    'target_energy': 0.3,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.35    
}

happy_params = {
    'min_acousticness': 0,
    'min_danceability': 0.5,
    'min_energy': 0.5,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
#     'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 60,
    'min_valence': 0.4,
    
    'max_acousticness': 1,
    'max_danceability': 1,
    'max_energy': 1,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 1,
    
#     'target_acousticness': 0.5,
    'target_danceability': 0.8,
    'target_energy': 0.8,
#     'target_instrumentalness': 0.5,
    'target_liveness': 0.3,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
#     'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.8    
}

sad_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
#     'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0,
    
    'max_acousticness': 1,
    'max_danceability': 0.4,
    'max_energy': 0.5,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
#     'max_popularity': 100,
    'max_speechiness': 1,
    'max_tempo': 120,
    'max_valence': 0.4,
    
    'target_acousticness': 0.6,
    'target_danceability': 0.2,
    'target_energy': 0.2,
    'target_instrumentalness': 0.8,
    'target_liveness': 0.3,
#     'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
    'target_tempo': 80,
    'target_valence': 0.2  
}

surprised_params = {
    'min_acousticness': 0.3,
    'min_danceability': 0.4,
    'min_energy': 0.3,
#     'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0.2,
    
    'max_acousticness': 1,
    'max_danceability': 0.6,
    'max_energy': 0.8,
    'max_instrumentalness': 1,
    'max_liveness': 0.4,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 1,
#     'max_tempo': 120,
    'max_valence': 1,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.5,
    'target_energy': 0.5,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
    'target_tempo': 108,
    'target_valence': 0.5    
}

neutral_params = {
    'min_acousticness': 0,
    'min_danceability': 0,
    'min_energy': 0.3,
    'min_instrumentalness': 0,
    'min_liveness': 0,
    'min_loudness': -60,
    'min_popularity': 0,
    'min_speechiness': 0,
    'min_tempo': 0,
    'min_valence': 0.3,
    
    'max_acousticness': 0.8,
    'max_danceability': 0.5,
    'max_energy': 0.7,
    'max_instrumentalness': 1,
    'max_liveness': 0.2,
    'max_loudness': 0,
    'max_popularity': 100,
    'max_speechiness': 0.6,
    'max_tempo': 140,
    'max_valence': 0.7,
    
    'target_acousticness': 0.5,
    'target_danceability': 0.4,
    'target_energy': 0.5,
    'target_instrumentalness': 0.5,
    'target_liveness': 0.5,
    'target_loudness': -60,
#     'target_popularity': 0.5,
    'target_speechiness': 0.5,
#     'target_tempo': 0.5,
    'target_valence': 0.45    
}

# emotion_labels = ['angry', 'disgusted', 'scared', 'happy', 'sad', 'surprised', 'neutral']

emotion_params = {'angry': angry_params, 'disgusted': disgusted_params, 'scared': scared_params, 'happy': happy_params, \
                  'sad': sad_params, 'surprised': surprised_params, 'neutral': neutral_params}

In [69]:
from utils import *
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\Antony\\Documents\\uChicago\\Winter 2022\\Music Thesis\\utils.py'>

In [70]:
pause_time = 10 # seconds

emotions_loop_mp = mp.Process(target=utils.music_loop, args = (data_q, pause_time, emotion_params, genre_overlap, 
                                                               user_top_songs, user_top_artists, user_country, sp))
emotions_loop_mp.start()

In [71]:
# Emergency stop to the loop if it becomes runaway
emotions_loop_mp.terminate()